In [2]:
import torch
from torch.autograd import Variable
from net import HGCTNet

In [5]:
model = HGCTNet(num_classes=2, num_hid=16, depth=2, num_heads=2, hga=True, trans=True)
sigs = Variable(torch.randn(5, 3, 12000))
feas = Variable(torch.randn(5, 53))
estbp = model(sigs, feas)